In [1]:
config_dir= '/home/amzad/Desktop/diffusion/config/config.yaml'
from Diffusion.utils.utils import read_config



config = read_config(config_dir)

print(config)

{'Train_config': {'train_name': 'diffusion unconditional', 'batch_size': 4, 'epochs': 500, 'lr': 0.0003, 'device': 'cuda', 'shaffle': True, 'intrable': 1, 'num_sample': 3, 'model_ckpt': '/home/amzad/Desktop/diffusion/artifacts/model_ckpt/', 'logs': '/home/amzad/Desktop/diffusion/logs/', 'figs': '/home/amzad/Desktop/diffusion/fig/', 'dataset': '/home/amzad/Desktop/diffusion/dataset/images/', 'ema_enable': True, 'ema': 0.999}, 'Noise_schedule': {'start': 0.0001, 'end': 0.02, 'steps': 1000, 'device': 'cuda'}, 'Dir': {'artifact_dir': '/home/amzad/Desktop/diffusion/artifacts'}, 'Transform_config': {'image_size': 64, 'resize': 80, 'crop': 64, 'mean': [0.5, 0.5, 0.5], 'std': [0.5, 0.5, 0.5], 'to_tensor': True}, 'model_config': {'cha_in': 3, 'cha_out': 3, 'time_dim': 256, 'device': 'cuda', 'selfattn': {'n_head': 4, 'batch_first': True}, 'dobule_conv': {'kernel_size': 3, 'stride': 1, 'padding': 1, 'bias': True}, 'down': {'kernel_size': 4, 'stride': 2, 'padding': 1, 'bias': False}}}


In [2]:
#__file__ = '/home/amzad/Desktop/diffusion/config/config.yaml'

In [3]:
import os 
import pathlib 
from pathlib import Path 


In [4]:
print(Path.home())
print(Path.cwd().parents[0])
print(Path.cwd().parents[0] / 'config' / 'config.yaml')

/home/amzad
/home/amzad/Desktop/diffusion
/home/amzad/Desktop/diffusion/config/config.yaml


In [5]:
test_config = Path.cwd().parents[0] / 'config' / 'config.yaml'
print(test_config)


/home/amzad/Desktop/diffusion/config/config.yaml


In [6]:
con = read_config(test_config) 

In [7]:
print(con)

{'Train_config': {'train_name': 'diffusion unconditional', 'batch_size': 4, 'epochs': 500, 'lr': 0.0003, 'device': 'cuda', 'shaffle': True, 'intrable': 1, 'num_sample': 3, 'model_ckpt': '/home/amzad/Desktop/diffusion/artifacts/model_ckpt/', 'logs': '/home/amzad/Desktop/diffusion/logs/', 'figs': '/home/amzad/Desktop/diffusion/fig/', 'dataset': '/home/amzad/Desktop/diffusion/dataset/images/', 'ema_enable': True, 'ema': 0.999}, 'Noise_schedule': {'start': 0.0001, 'end': 0.02, 'steps': 1000, 'device': 'cuda'}, 'Dir': {'artifact_dir': '/home/amzad/Desktop/diffusion/artifacts'}, 'Transform_config': {'image_size': 64, 'resize': 80, 'crop': 64, 'mean': [0.5, 0.5, 0.5], 'std': [0.5, 0.5, 0.5], 'to_tensor': True}, 'model_config': {'cha_in': 3, 'cha_out': 3, 'time_dim': 256, 'device': 'cuda', 'selfattn': {'n_head': 4, 'batch_first': True}, 'dobule_conv': {'kernel_size': 3, 'stride': 1, 'padding': 1, 'bias': True}, 'down': {'kernel_size': 4, 'stride': 2, 'padding': 1, 'bias': False}}}


In [11]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision.models as models

class ScalingLayer(nn.Module):
    def __init__(self):
        super(ScalingLayer, self).__init__()
        self.register_buffer('shift', torch.Tensor([-.030, -.088, -.188])[None, :, None, None])
        self.register_buffer('scale', torch.Tensor([.458, .448, .450])[None, :, None, None])

    def forward(self, x):
        return (x - self.shift) / self.scale

class NetLinLayer(nn.Module):
    def __init__(self, ch, use_dropout=True):
        super(NetLinLayer, self).__init__()
        layers = [nn.Conv2d(ch, 1, 1, 1, 0, bias=False)]
        if use_dropout:
            layers.append(nn.Dropout())
        self.model = nn.Sequential(*layers)

    def forward(self, x):
        return self.model(x)

class LPIPS(nn.Module):
    def __init__(self, net="vgg", use_dropout=True):
        super(LPIPS, self).__init__()
        self.scaling_layer = ScalingLayer()
        vgg = models.vgg16(pretrained=True).features
        self.layers = [3, 8, 15, 22, 29]
        self.vgg = nn.Sequential(*[vgg[i] for i in range(max(self.layers) + 1)])
        self.lins = nn.ModuleList([NetLinLayer(ch, use_dropout=use_dropout) for ch in [64, 128, 256, 512, 512]])
        self.eval()
        for param in self.parameters():
            param.requires_grad = False

    def forward(self, in0, in1, normalize=False):
        if normalize:
            in0, in1 = 2 * in0 - 1, 2 * in1 - 1

        in0_input, in1_input = self.scaling_layer(in0), self.scaling_layer(in1)
        outs0, outs1 = self.vgg(in0_input), self.vgg(in1_input)
        
        feats0, feats1, diffs = [], [], []
        for i, layer in enumerate(self.layers):
            feats0.append(F.normalize(outs0[layer], dim=1))
            feats1.append(F.normalize(outs1[layer-1], dim=1))
            diffs.append((feats0[-1] - feats1[-1]) ** 2)

        res = [F.adaptive_avg_pool2d(self.lins[i](diffs[i]), (1, 1)) for i in range(len(self.layers))]
        return sum(res)
    
    def __str__(self):
        return F"VGG16 with LPIPS loss {self.lins} layers and {self.vgg} features  "
    

if __name__ == '__main__':
    lpips = LPIPS()
    x = torch.randn(1, 3, 256, 256)
    y = torch.randn(1, 3, 256, 256)
    L = lpips(x, y) 
    print(L) 


IndexError: index 3 is out of bounds for dimension 0 with size 1